In [ ]:
# Importing dependencies
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import random
from matplotlib.animation import FuncAnimation

def generate_plot(graph):
    fig, ax = ox.plot_graph(graph, node_color='blue', edge_linewidth=2, bgcolor='white', show=False)

    # Add label with road length (m)
    for u, v, key, data in graph.edges(keys=True, data=True):
        if 'length' in data:
            # Compute the midpoint of the edge
            x1, y1 = graph.nodes[u]['x'], graph.nodes[u]['y']
            x2, y2 = graph.nodes[v]['x'], graph.nodes[v]['y']
            mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
            
            # Round the length to whole number
            label = str(round(data['length']))
            ax.text(mid_x, mid_y, label, color='black', size=10, ha='center', va='center')

def animate_graph_traversal(graph, traversal_path, filename='traversal.gif'):
    fig, ax = ox.plot_graph(graph, node_color='none', edge_color='gray', show=False, close=False)

    # A dictionary to store the count of traversals for each edge
    edge_counts = {(u, v): 0 for u, v in graph.edges()}

    # The colormap to convert traversal counts to colors
    cmap = plt.cm.plasma

    def update(frame):
        u, v = traversal_path[frame]

        # Increase the count for the edge
        if (u, v) in edge_counts:
            edge_counts[(u, v)] += 1
        else:
            edge_counts[(v, u)] += 1

        # Determine the color for the edge based on its traversal count
        max_count = max(edge_counts.values())
        if (u, v) in edge_counts:
            color = cmap(edge_counts[(u, v)] / max_count)
        else:
            color = cmap(edge_counts[(v, u)] / max_count)

        # Plot the edge with the determined color
        ox.plot_graph_route(graph, [u, v], route_linewidth=2, node_size=0, bgcolor='white', route_color=color, ax=ax)

    ani = FuncAnimation(fig, update, frames=len(traversal_path), repeat=False)

    # Save the animation as GIF
    ani.save(filename, writer='pillow', fps=3)

    plt.close(fig)

def compare_graphs_by_connected_edges(graph1, graph2):
    # Compare edge sets
    if set(graph1.edges()) == set(graph2.edges()):
        print('[OK] Graphs have identical connected edges')
        return

    print('[FAIL] Graphs DO NOT have identical connected edges')


In [ ]:
# Download the road network graph
lat, lon = 56.958141067669146, 24.12462850613549
directed_graph = ox.graph_from_point((lat, lon), dist=400, network_type='drive') # Get surrounding area in N meter radius

In [ ]:
# Remove road directions to get undirected graph
graph = directed_graph.to_undirected()

In [ ]:
# Select largest connected graph
largest_connected_component = max(nx.connected_components(graph), key=len)
graph = graph.subgraph(largest_connected_component).copy()

In [ ]:
# Generate plot to verify that graph is suitable
generate_plot(graph)

In [ ]:
# Given two nodes u and v from source_graph, copy the edge and its attributes to target_graph
# Update path_length, traversed_edges and solution_path
def copy_edge_to_solution(u, v, source_graph, target_graph):
    global path_length, solution_path, traversed_edges

    solution_path.append((u, v))
    traversed_edges.add((u, v))
    traversed_edges.add((v, u))

    path_length += source_graph[u][v][0]['length']

    data = source_graph[u][v][0]
    target_graph.add_edge(u, v, **data)

In [ ]:
path_length = 0
solution_path = []
traversed_edges = set()

# Generate initial (likely unoptimal) feasible solution
def generate_initial_solution(graph):
    global weight, solution_path, traversed_edges

    # Configure solution graph
    solution_graph = nx.MultiDiGraph() # Directed graph
    solution_graph.graph['crs'] = graph.graph['crs']
    solution_graph.add_nodes_from(graph.nodes(data=True)) # Copy all nodes (without edges)

    # Start from a random vertex
    # TODO: choice could be optimized
    start_vertex = random.choice(list(graph.nodes))
    edge_count = graph.number_of_edges()

    # Keep track of path to backtrack our steps
    path = [start_vertex]

    # While there are nodes in path and while not all graph edges were traversed
    while path and len(traversed_edges) < 2 * edge_count:
        current_vertex = path[-1] # get the most recent node in path

        neighbor_vertices = list(graph.neighbors(current_vertex))
        unvisited_neighbors = [n for n in neighbor_vertices if (current_vertex, n) not in traversed_edges and (n, current_vertex) not in traversed_edges]

        if unvisited_neighbors:
            # Get neighbor with the smallest amount of neighbors
            sorted_neighbors = sorted(unvisited_neighbors, key=lambda x: len(list(graph.neighbors(x))))
            next_vertex = sorted_neighbors[0]

            # Add the edge to the solution graph
            copy_edge_to_solution(current_vertex, next_vertex, graph, solution_graph)

            # Debugging
            # fig, ax = ox.plot_graph(solution_graph, node_color='gray', edge_linewidth=2, bgcolor='white', show=True)

            path.append(next_vertex)
        else:
            path.pop()  # backtrack
            backstep_vertex = path[-1]

            if path:
                # Add the edge to the solution graph
                copy_edge_to_solution(current_vertex, backstep_vertex, graph, solution_graph)

    return solution_graph, solution_path

solution, traversed_path = generate_initial_solution(graph)
compare_graphs_by_connected_edges(graph, solution.to_undirected())
animate_graph_traversal(graph, traversed_path)
print(path_length)
# print(solution_path)

In [ ]:
# Compare solution against initial graph, to verify that all edges were indeed traversed
generate_plot(graph)
generate_plot(solution)